In [14]:
!pip install spacy --upgrade
!python -m spacy download en
import spacy
nlp = spacy.load('en')

Requirement already up-to-date: spacy in /opt/conda/lib/python3.7/site-packages (2.2.4)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/conda/lib/python3.7/site-packages/en_core_web_sm -->
/opt/conda/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [15]:
def customize_stop_words(nlp):
  token_exception_list = ['by', 'per', 'how', 'many', 'much', 'and', 'most', 'more', 'than', 'hundred']    
  for token in token_exception_list:
    if nlp.vocab[token].is_stop:
      nlp.vocab[token].is_stop = False

In [17]:
from spacy import displacy
def customize_displacy(doc):
  displacy.render(doc, style='dep', jupyter=True, options={'distance': 90, 'fine_grained' : True, 'add_lemma' : True, 'collapse_phrases' : False})
  


In [7]:
doc = nlp('How many oil fields do exist?')
for token in doc:
  print(token.dep_)
customize_displacy(doc)



advmod
amod
compound
nsubj
aux
ROOT
punct


In [17]:
from spacy import displacy
import spacy
from spacy.matcher import Matcher

class Glamorise:

  # instance attribute
  def __init__(self, txt, lang = "en"):
    self.__nlp = spacy.load(lang)
    self.__doc = self.__nlp(txt)
    self.__customize_stop_words
    self.__aggregate_function = ''
    self.__aggregate_field = ''
    self.__group_by_field = ''
    self.__having_field = ''
    self.__cut_text = []
    self.__group_by = False
    self.__having = ''
    self.__matcher = Matcher(self.__nlp.vocab)

  @property
  def aggregate_function(self):
    return self.__aggregate_function
  
  @property
  def aggregate_field(self):
    return self.__aggregate_field
  
  @property
  def group_by_field(self):
    return self.__group_by_field

  @property
  def having_field(self):
    return self.__having_field
  @property
  def cut_text(self):
    return self.__cut_text

  @property
  def group_by(self):
    return self.__group_by

  @property
  def having(self):
    return self.__having
    
  def __customize_stop_words(self):
    token_exception_list = ['by', 'per', 'how', 'many', 'much', 'and', 'more', 'greater', 'than', 'hundred', 'of']    
    for token in token_exception_list:
      if self.__nlp.vocab[token].is_stop:
        self.__nlp.vocab[token].is_stop = False      
  
  def matcher(self, patterns):        
    for pattern in patterns:      
      self.__matcher.add(str(patterns), None, pattern)    
      matches = self.__matcher(self.__doc)
      if matches != []:
        return True 
    return False
      

  def build_field(self, token, type):    
    print("token.ancestors", str(token.ancestors))
    for ancestor in token.ancestors:            
      # set the field
      field = ancestor.lemma_        
      accum = ''                  
      #look for compund terms
      for ancestor_children in ancestor.subtree:                                          
        # get compound terms that are not proper noun
        if ancestor_children.dep_ == 'compound' and ancestor_children.tag_ != 'NNP':                
          accum = accum + ancestor_children.lemma_ + ' '                         
      field = accum + field          
      if type == 'aggregate':
        self.__aggregate_field = field
      if type == 'group by':
        self.__group_by_field = field
      if type == 'having':        
        self.__having_field = field
      break
    
  '''
  def pattern_match(self, token, reserved_words, children_reserved_words = None, ancestors_reserved_words = None): 
    if token.lemma_ in reserved_words:              
        if children_reserved_words is None and ancestors_reserved_words is None:
          print("reserved_words")
          return True
        children = list(token.children)
        print("children")
        for child in children:                
          if child.lemma_ in children_reserved_words:       
            if ancestors_reserved_words is None:
              return True
            else:
              break  
        ancestors = list(token.ancestors)
        print("parent")
        for ancestor in ancestors:                
          if ancestor.lemma_ in ancestor_reserved_words:                   
            return True
    return False
  '''
  def pattern_match(self, token, reserved_words, children_reserved_words = None, group_by = False, having = ''):     
    if token.lemma_ in reserved_words:      
      if reserved_words[token.lemma_] is not None:
        self.__aggregate_function = reserved_words[token.lemma_]      
      self.__group_by = group_by
      self.__having = having
      if not group_by and having == '':
        self.build_field(token, type = 'aggregate')
      elif group_by:
        self.build_field(token, type = 'group by')
      elif having != '':
        self.build_field(token, type = 'having')
      if children_reserved_words is None:                
        self.__cut_text.append(token.lemma_)
      else:  
        children = list(token.children)        
        for child in children:             
          if child.lemma_ in children_reserved_words:                                         
            self.__cut_text.append(child.lemma_ + ' ' + token.lemma_)
          
          
      

  def pattern_scan(self):     
    for token in self.__doc:        
      '''
      # get the how many, how much pattern
      if token.lemma_ in ['many', 'much']:      
        children = list(token.children)      
        for child in children:                
          if child.lemma_ == 'how':       
            #set aggregate_function
            if token.lemma_ == 'many':
              self.__aggregate_function = 'count'  
            elif  token.lemma_ == 'much':
              self.__aggregate_function = 'sum'           
            #set the text that will be removed from the query before passing to the NLIDB
            self.__cut_text = child.lemma_ + ' ' + token.lemma_          
            self.build_aggregate_field(token)
      '''      
      # get the how many, how much pattern
      self.pattern_match(token, {'many' : 'count', 'number' : 'count', 'much' : 'sum'}, ['how', 'of'])       

      # get the most, maximum, max pattern
      self.pattern_match(token, {'most' : 'max', 'maximum' : 'max', 'max' : 'max'})  

      # get the least, minimum, min pattern
      self.pattern_match(token, {'least': 'min', 'minimum' : 'min', 'min' : 'min'})

      # get the average, mean pattern
      self.pattern_match(token, {'average' : 'avg', 'mean' : 'avg', 'avg' : 'avg'})

      # get the by, per pattern
      self.pattern_match(token, {'by' : None , 'per' : None}, group_by = True)     

      # get the greater than, more than pattern
      self.pattern_match(token, {'great' : None, 'more' : None}, having = '+') 

      # get the less than pattern
      self.pattern_match(token, {'less' : None}, having = '-') 

      # get the equal to pattern
      self.pattern_match(token, {'equal' : None}, having = '=') 
  
  def customized_displacy(self):
    displacy.render(self.__doc, style='dep', jupyter=True, options={'distance': 90, 'fine_grained' : True, 'add_lemma' : True, 'collapse_phrases' : False})

ModuleNotFoundError: No module named 'pydev_jupyter_utils'

In [17]:
import csv

with open('./datasets/teste.csv') as csv_file:
#with open('/content/sample_data/teste.csv') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',', quotechar="'")
  line_count = 0
  for row in csv_reader:    
    nl_query = row[0]
    print("\n\nNatural Language Query: ",nl_query )
    glamorise = Glamorise(nl_query)
    glamorise.pattern_scan()
    glamorise.customized_displacy()
    print("aggregate function: ", glamorise.aggregate_function)
    print("aggregate field: ", glamorise.aggregate_field)
    print("group_by_field: ", glamorise.group_by_field)
    print("having_field: ", glamorise.having_field)
    print("group_by: ", glamorise.group_by)
    print("having: ", glamorise.having)
    print("cut_text: ", glamorise.cut_text)





ModuleNotFoundError: No module named 'pydev_jupyter_utils'

# New Section

# New Section

In [11]:
myList = {'a': 1, 'b' : 2,'c': 5}
'a' in myList


True

In [12]:
import nltk
print(nltk.__version__)

nltk.download('wordnet')
from nltk.corpus import wordnet as wn
 
# Just to make it a bit more readable
WN_NOUN = 'n'
WN_VERB = 'v'
WN_ADJECTIVE = 'a'
WN_ADJECTIVE_SATELLITE = 's'
WN_ADVERB = 'r'
 
def convert(word, from_pos, to_pos):    
    """ Transform words given from/to POS tags """
 
    synsets = wn.synsets(word, pos=from_pos)
    print(synsets)
 
    # Word not found
    if not synsets:
        return []
    
    # Get all lemmas of the word (consider 'a'and 's' equivalent)
    lemmas = [l for s in synsets
                for l in s.lemmas()
                if s.name().split('.')[1] == from_pos
                    or from_pos in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)
                        and s.name().split('.')[1] in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)]
    print("lemmas",lemmas)
    # Get related forms
    derivationally_related_forms = [(l, l.derivationally_related_forms()) for l in lemmas]
    print("derivationally_related_forms",derivationally_related_forms)
    # return derivationally_related_forms
    # filter only the desired pos (consider 'a' and 's' equivalent)
    related_noun_lemmas = [l for drf in derivationally_related_forms
                             for l in drf[1] 
                             if l.synset().name().split('.')[1] == to_pos
                                or to_pos in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)
                                    and l.synset().name().split('.')[1] in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)]
 
    # Extract the words from the lemmas
    words = [l.name() for l in related_noun_lemmas]    
    len_words = len(words)
 
    # Build the result in the form of a list containing tuples (word, probability)
    result = [(w, float(words.count(w))/len_words) for w in set(words)]
    result.sort(key=lambda w: -w[1])
 
    # return all the possibilities sorted by probability
    return result

ModuleNotFoundError: No module named 'nltk'

In [ ]:
print(convert("productive", WN_ADJECTIVE, WN_NOUN))

In [ ]:
synsets = wn.synsets('productive', 'a')
from_pos = 'a'
# Get all lemmas of the word (consider 'a'and 's' equivalent)
lemmas = [l for s in synsets
            for l in s.lemmas()
            if s.name().split('.')[1] == from_pos
                or from_pos in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)
                    and s.name().split('.')[1] in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)]

# Get related forms
derivationally_related_forms = [(l, l.derivationally_related_forms()) for l in lemmas]

# filter only the desired pos (consider 'a' and 's' equivalent)
for drf in derivationally_related_forms:
  for l in drf[1]:
    print(l)
            #              if l.synset().name().split('.')[1] == to_pos
            #                or to_pos in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)
              #                   and l.synset().name().split('.')[1] in (WN_ADJECTIVE, WN_ADJECTIVE_SATELLITE)]